# Trains and Tests PLDA

In [558]:
#!/usr/bin/env python3.7
import os
import sys
sys.path.insert(0, "/export/c10/lmorove1/PythonLibs/plda")
import plda
import numpy as np
import matplotlib.pyplot as plt
import kaldi_io
import argparse
import pickle
from sklearn.decomposition import PCA

import pandas as pd 

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error

#PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Confusion matrix
from sklearn import metrics

In [425]:
DATA_PATH = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/'

!pwd && ls
os.chdir(DATA_PATH)
!pwd && ls

/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     pca_pldapy.ipynb  runFor.sh  run.sh~  steps
cmd.sh~  data  local  path.sh  README.txt	 run.sh     sid      utils
/export/c08/lmorove1/kaldi/egs/beatPDivec/v1
cmd.sh	 conf  exp    mfcc     pca_pldapy.ipynb  runFor.sh  run.sh~  steps
cmd.sh~  data  local  path.sh  README.txt	 run.sh     sid      utils


In [426]:
def pca(sFileTrai, sFileTest, iComponents):
# os.chdir("/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k")
# sFileTrai="/export/c10/lmorove1/kaldi/egs/xVecPD/v2_NeuroConc16k/exp/sre18nn/xvectors_Training_Fold1/xvector.scp"
# sOut = Folder where to store results 
# iComponents = nb of classes (nb of measurements_id? or nb of subject_ids?)
    dIvecTrai = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTrai) }
    vTrai= pd.DataFrame((list(dIvecTrai.values())))
    vLTrai = np.array([x[-1] for x in np.array(list(dIvecTrai.keys()))])
    
    # FIXME : For realPD, we need more than -5 (CIS-PD subject_id is 4 characters long)
    # FIXME REAL-PD it's not only int 
    vTraiSubjectId = np.array(([int(x[-5:-1]) for x in np.array(list(dIvecTrai.keys()))]))

    dIvecTest = { key:mat for key,mat in kaldi_io.read_vec_flt_scp(sFileTest) }
    vTest=np.array(list(dIvecTest.values()), dtype=float)
    vLTest=np.array([x[-1] for x in np.array(list(dIvecTest.keys()))])
    vTestSubjectId = np.array([int(x[-5:-1]) for x in np.array(list(dIvecTest.keys()))])

    print('before transform : ', vTrai.shape)
    print('before transform : ', vTest.shape)

    if isinstance(iComponents, str):
            iComponents=int(iComponents)

    pca = PCA(n_components=iComponents, svd_solver='randomized', whiten=True)
    pca.fit(vTrai)


    vTraiPCA=pca.transform(vTrai)
    vTestPCA=pca.transform(vTest)

    print('after transform : ', vTraiPCA.shape)
    print('after transform : ', vTestPCA.shape)
    return vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId

In [427]:
sFileTrai = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Training_Fold0/ivector.scp'
sFileTest = '/export/c08/lmorove1/kaldi/egs/beatPDivec/v1/exp/ivectors_Testing_Fold0/ivector.scp'
iComponents = 50 


In [553]:
vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId = pca(sFileTrai, sFileTest, iComponents)


before transform :  (1403, 300)
before transform :  (364, 300)
after transform :  (1403, 50)
after transform :  (364, 50)


### Without GridSearch 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# To compute the final score as per the challenge
mse_training_per_subjectid=[]
mse_test_per_subjectid=[]
train_nb_files_per_subjectid=[]
test_nb_files_per_subjectid=[]

# To compute the mean accuracy
glob_trai_pred=[]
glob_test_pred=[]
glob_trai_true=[]
glob_test_true=[]
    
for subject_id in np.unique(vTraiSubjectId):
    # GridSearchCV
#     parameters = {'n_neighbors':[1,3,6,10,15]}

    k_range = list(range(1, 15))
#     param_grid = dict(n_neighbors=k_range)
#     print(param_grid)

    # instantiate the grid
#     grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')

    # fit the grid with data
#     grid.fit(vTraiPCA_subjectid, vLTrai_subjectid)

    # view the complete results (list of named tuples)
#     print(grid.cv_results_['mean_test_score'])

    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=6)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
#     y_labels = knn.predict(vTraiPCA_subjectid)
#     display(y_labels)

    print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    
    # Predicting on the training and test data 
    predictionsTrai = knn.predict(vTraiPCA_subjectid)
    predictions = knn.predict(vTestPCA_subjectid)
    
    # Converting all strings to int for MSE 
    vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
    predictionsTrai = [int(i) for i in predictionsTrai]
    vLTest_subjectid = [int(i) for i in vLTest_subjectid]
    predictions = [int(i) for i in predictions]
 
    # Computing the accuracy 
    glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
    glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
    glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
    glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)
    
    a = [a - b for a, b in zip(vLTrai_subjectid, predictionsTrai)]
    print('abs substract : ', np.abs(a))
    
    # Building a list of the MSEk 
    mse_training_per_subjectid = np.append(mse_training_per_subjectid,
                                           (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
    mse_test_per_subjectid = np.append(mse_test_per_subjectid,
                                        (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
    train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
    test_nb_files_per_subjectid.append(len(vLTest_subjectid))

#     do_confusion_matrix(vLTest_subjectid, predictions)

print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))

### Tentative with GridSearch 

In [576]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# To compute the final score as per the challenge
mse_training_per_subjectid=[]
mse_test_per_subjectid=[]
train_nb_files_per_subjectid=[]
test_nb_files_per_subjectid=[]

# To compute the mean accuracy
glob_trai_pred=[]
glob_test_pred=[]
glob_trai_true=[]
glob_test_true=[]
    
for subject_id in np.unique(vTraiSubjectId):
    # GridSearchCV
#     parameters = {'n_neighbors':[1,3,6,10,15]}
    knn2 = KNeighborsClassifier()
    k_range = list(range(1, 15))
    param_grid = dict(n_neighbors=k_range)
#     print(param_grid)

    # instantiate the grid
    grid = GridSearchCV(knn2, param_grid, cv=5, scoring='accuracy')

    # fit the grid with data
    grid.fit(vTraiPCA_subjectid, vLTrai_subjectid)

    #check top performing n_neighbors value
    print(grid.best_params_)
    
    #print(grid.cv_results_)
    #check mean score for the top performing value of n_neighbors
    print(grid.best_score_)

    print('----- ' + str(subject_id) + '----- ')
    knn = KNeighborsClassifier(n_neighbors=7)
    
    # Filter vTraiPCA and vLTraiPCA for one subject_id
    indices_subject_id = np.where(vTraiSubjectId == subject_id) # HAPPY
    vTraiPCA_subjectid = vTraiPCA[indices_subject_id]
    vLTrai_subjectid = vLTrai[indices_subject_id]
    
    # Filter vTestPCA and vLTestPCA for one subject_id
    indices_subject_id = np.where(vTestSubjectId == subject_id)
    vTestPCA_subjectid = vTestPCA[indices_subject_id]
    vLTest_subjectid = vLTest[indices_subject_id]

    knn.fit(vTraiPCA_subjectid, vLTrai_subjectid)
    
#     y_labels = knn.predict(vTraiPCA_subjectid)
#     display(y_labels)

    print('Training accuracy: ', knn.score(vTraiPCA_subjectid, vLTrai_subjectid))
    print('Testing accuracy: ', knn.score(vTestPCA_subjectid, vLTest_subjectid))
    
    # Predicting on the training and test data 
    predictionsTrai = knn.predict(vTraiPCA_subjectid)
    predictions = knn.predict(vTestPCA_subjectid)
    
    # Converting all strings to int for MSE 
    vLTrai_subjectid = [int(i) for i in vLTrai_subjectid]
    predictionsTrai = [int(i) for i in predictionsTrai]
    vLTest_subjectid = [int(i) for i in vLTest_subjectid]
    predictions = [int(i) for i in predictions]
 
    # Computing the accuracy 
    glob_trai_pred=np.append(glob_trai_pred,predictionsTrai,axis=0)
    glob_test_pred=np.append(glob_test_pred,predictions,axis=0)
    glob_trai_true=np.append(glob_trai_true,vLTrai_subjectid,axis=0)
    glob_test_true=np.append(glob_test_true,vLTest_subjectid,axis=0)
    
    a = [a - b for a, b in zip(vLTrai_subjectid, predictionsTrai)]
    print('abs substract : ', np.abs(a))
    
    # Building a list of the MSEk 
    mse_training_per_subjectid = np.append(mse_training_per_subjectid,
                                           (mean_squared_error(vLTrai_subjectid, predictionsTrai) /  len(vLTrai_subjectid)))
    mse_test_per_subjectid = np.append(mse_test_per_subjectid,
                                        (mean_squared_error(vLTest_subjectid, predictions) / len(vLTest_subjectid)))
    train_nb_files_per_subjectid.append(len(vLTrai_subjectid))
    test_nb_files_per_subjectid.append(len(vLTest_subjectid))

#     do_confusion_matrix(vLTest_subjectid, predictions)

print('Global training accuracy: {}'.format((glob_trai_true == glob_trai_pred).mean()))
print('Global testing accuracy: {}'.format((glob_test_true == glob_test_pred).mean()))

/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-pack

{'n_neighbors': 7}
0.7402597402597403
----- 1004----- 
Training accuracy:  0.36923076923076925
Testing accuracy:  0.29411764705882354
abs substract :  [0 1 0 0 3 1 0 2 2 1 1 0 0 0 1 3 1 0 2 3 3 1 0 0 2 1 2 1 2 1 3 0 1 3 2 1 1
 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 3 1 0 1 0 2 0 4 1 0]
{'n_neighbors': 2}
0.24615384615384617
----- 1006----- 
Training accuracy:  0.7586206896551724
Testing accuracy:  0.625
abs substract :  [0 0 0 0 0 0 0 0 1 1 0 1 0 0 2 0 0 1 0 0 0 1 0 0 0 0 1 0 0]


/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 6}
0.7586206896551724
----- 1007----- 
Training accuracy:  0.5833333333333334
Testing accuracy:  0.5166666666666667
abs substract :  [2 0 0 0 1 1 1 4 1 0 1 4 0 2 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 2 0 0 2 1 0 0
 3 0 2 2 0 1 0 1 0 0 2 0 0 2 0 2 0 0 2 4 0 1 1 1 0 0 0 0 1 4 0 2 0 0 0 0 0
 0 1 0 0 0 3 0 0 0 1 0 4 1 0 0 0 0 0 0 1 2 0 0 0 0 0 0 1 2 3 0 0 2 0 3 0 0
 4 3 0 0 3 1 2 1 0 4 1 0 1 0 2 0 0 0 0 0 0 0 4 0 0 1 2 4 0 1 0 1 0 0 0 0 4
 3 0 1 1 0 0 0 0 0 4 2 1 0 0 0 0 4 0 2 2 1 0 0 1 0 3 0 0 0 0 0 3 0 1 0 0 0
 0 1 0 0 0 0 0 0 3 1 0 2 0 1 3 0 0 1 1 2 0 0 2 0 0 1 1 4 0 0 1]


/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


...

{'n_neighbors': 6}
0.46153846153846156
----- 1043----- 
Training accuracy:  0.5555555555555556
Testing accuracy:  0.14285714285714285
abs substract :  [2 0 0 2 0 0 0 0 2 0 2 0 1 0 0 2 2 0 3 0 3 2 0 1 3 0 0]
{'n_neighbors': 5}
0.3333333333333333
----- 1044----- 
Training accuracy:  0.6666666666666666
Testing accuracy:  0.6666666666666666
abs substract :  [1 1 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0]


/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_split.py:626: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'n_neighbors': 4}
0.5964912280701754
----- 1048----- 
Training accuracy:  0.6111111111111112
Testing accuracy:  0.3157894736842105
abs substract :  [1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 2 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 2 1 1
 0 0 0 0 2 0 3 0 0 1 0 1 1 0 2 2 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0]


/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/mpgill/anaconda3/envs/BeatPD/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 14}
0.5555555555555556
----- 1049----- 
Training accuracy:  0.5384615384615384
Testing accuracy:  0.6470588235294118
abs substract :  [0 0 0 2 2 0 0 1 1 0 0 0 1 0 2 0 3 0 2 0 0 2 0 0 0 0 2 0 2 0 0 0 1 2 1 0 0
 1 1 1 0 1 0 0 1 2 0 1 2 0 3 2 0 2 0 1 1 0 0 0 0 1 1 2 0]
{'n_neighbors': 14}
0.4153846153846154
----- 1051----- 
Training accuracy:  0.7402597402597403
Testing accuracy:  0.6666666666666666
abs substract :  [1 1 3 2 0 0 0 0 1 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0
 4 0 0 0 0 0 1 0 0 0 0 0 0 0 3 1 0 3 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0
 3 2 0 0 2 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 1 0 2 2 2 0 1 0
 0 0 0 4 0 0 0 0 1 4 0 0 0 0 2 0 0 0 0 0 3 2 0 0 3 2 4 2 0 3 0 0 0 3 0 0 0
 4 0 0 0 0 0]
Global training accuracy: 0.6065573770491803
Global testing accuracy: 0.5357142857142857


In [575]:
from sklearn.metrics import mean_squared_error
from math import sqrt 

def final_score(mse_per_subjectid, nb_files_per_subject_id):
    numerator = np.sum([a * b for a, b in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
    denominator = np.sum(np.sqrt(nb_files_per_subject_id))
    print('numerator : ', numerator)
    print('denominator : ', denominator)
    print('Final score : ', np.divide(numerator, denominator))

In [556]:
final_score(mse_training_per_subjectid, train_nb_files_per_subjectid)
final_score(mse_test_per_subjectid, test_nb_files_per_subjectid)

numerator :  2.9956883253194277
denominator :  138.07717596772696
Final score :  0.021695753149091168
numerator :  6.065415875730853
denominator :  70.2868377625761
Final score :  0.08629518795851071


In [543]:
def do_confusion_matrix(y_test, predictions):
    print('y test : ', np.unique(y_test))
    LABELS_NEW = np.unique(y_test)
    n_classes=np.unique(y_test)
    print("Confusion Matrix:")
    confusion_matrix = metrics.confusion_matrix(y_test, predictions)
    print(confusion_matrix)
    normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

    print("")
    print("Confusion matrix (normalised to % of total test data):")
    print(normalised_confusion_matrix)

    # Plot Results:
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    plt.imshow(
        normalised_confusion_matrix,
        interpolation='nearest',
        cmap=plt.cm.rainbow
    )
    plt.title("Confusion matrix \n(normalised to % of total test data)")
    plt.colorbar()
    tick_marks = np.arange(n_classes)
    plt.xticks(tick_marks, LABELS_NEW, rotation=90)
    plt.yticks(tick_marks, LABELS_NEW)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()